# Data Loading and Splitting

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import numpy as np
import time

import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt

In [0]:
# Loading Gesture Images from Google Drive

# location on Google Drive
orig_path = '/content/gdrive/My Drive/APS360/APS360 Final Project/Dataset/dataset2/images/TRAIN'
test_path = '/content/gdrive/My Drive/APS360/APS360 Final Project/Dataset/dataset2/images/TEST'

# Transform Settings - Do not use RandomResizedCrop
transform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])

orig_set = torchvision.datasets.ImageFolder(orig_path, transform=transform)

# val_set = torch.utils.data.Subset(orig_set, range(a))  
# train_set = torch.utils.data.Subset(orig_set, range(a, b))   
n = len(orig_set)
n_val = int(0.25*n)
val_set, train_set = torch.utils.data.random_split(orig_set, [n_val, n-n_val])

test_set = torchvision.datasets.ImageFolder(test_path, transform=transform)

print("The length of train set: {}".format(len(train_set)))
print("The length of validation set: {}".format(len(val_set)))
print("The length of test set: {}".format(len(test_set)))


# Prepare Dataloader
batch_size = 32
num_workers = 1
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)


# Verification Step - obtain one batch of images
dataiter = iter(val_loader)
images, labels = dataiter.next()
images = images.numpy() # convert images to numpy for display

classes = ['EOSINOPHIL', 'LYMPHOCYTE', 'MONOCYTE', 'NEUTROPHIL']

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    plt.imshow(np.transpose(images[idx], (1, 2, 0)))
    ax.set_title(classes[labels[idx]])


# Data Processing: Finding Bounding Box

In [0]:
# import the original images and labels

import cv2
import csv
import re
from PIL import Image
import os, os.path
from matplotlib.image import imread


# get images, labels from original data
orig_dataset_path = '/content/gdrive/My Drive/APS360/Dataset/dataset1/'
img_path = orig_dataset_path + 'JPEGImages/'
label_path = orig_dataset_path + 'labels.csv'


# with open(label_path) as f:
#     reader = csv.reader(f)
#     data = [tuple(row) for row in reader]
#     data = data[1:]

# # create label dictionary for each image
# # label[img_index] = label for image with id img_index
# all_labels = {}
# for i in range(len(data)):
#     all_labels[int(data[i][1])] = data[i][2].split(',')

# for i in range(len(data)):
#     if(len(all_labels[i]) > 1):
#         for j in range(len(all_labels[i])):
#             all_labels[i][j] = all_labels[i][j].strip()


# imgs[img_index] = img with id img_index
labels = {}
imgs = {}
for f in os.listdir(img_path):
    index = int(re.findall('\d+', f)[0])
    imgs[index] =  imread(img_path + f)
    # labels[index] = all_labels[index]

print(len(imgs))
print(len(labels))

# i = 0
# print(labels[i])
# plt.imshow(imgs[i])

In [0]:
# functions to find wbc bounding boxes in an img

def distance(x1, y1, x2, y2):
    return ((x2-x1)**2 + (y2-y1)**2) ** 0.5

def find_min_distance(rect1,rect2):
  dist = []
  
  dist.append(distance(rect1[0],rect1[1],rect2[0]+rect2[2]/2,rect2[1]+rect2[3]/2))
  dist.append(distance(rect1[0]+rect1[2],rect1[1],rect2[0]+rect2[2]/2,rect2[1]+rect2[3]/2))
  dist.append(distance(rect1[0],rect1[1]+rect1[3],rect2[0]+rect2[2]/2,rect2[1]+rect2[3]/2))
  dist.append(distance(rect1[0]+rect1[2],rect1[1]+rect1[3],rect2[0]+rect2[2]/2,rect2[1]+rect2[3]/2))
  return min(dist)


def merge_by_iou(rects):
    clusters = []
    merge = 1
    used_rect_list = []

    for i in range(len(rects)):
        if not i in used_rect_list:
            temp = []
            temp.append(rects[i])
            used_rect_list.append(i)
            for j in range(i+1,len(rects)):
                if not j in used_rect_list:
                    iou = iou_val(
                        rects[i][0], rects[i][0] + rects[i][2], rects[i][1], rects[i][1] + rects[i][3], 
                        rects[j][0], rects[j][0] + rects[j][2], rects[j][1], rects[j][1] + rects[j][3])
                    if iou > 0.9:
                        temp.append(rects[j])
                        used_rect_list.append(j)
            clusters.append(temp)

                
    output = []
    for i in range(len(clusters)):
      lx, ly, rx, ry = 10000, 10000, -10000, -10000
      for j in range(len(clusters[i])):
        if lx > clusters[i][j][0]:
          lx = clusters[i][j][0]
        if ly > clusters[i][j][1]:
          ly = clusters[i][j][1]
        if rx < clusters[i][j][0]+clusters[i][j][2]:
          rx = clusters[i][j][0]+clusters[i][j][2]
        if ry < clusters[i][j][1]+clusters[i][j][3]:
          ry = clusters[i][j][1]+clusters[i][j][3]
      output.append((lx,ly,rx-lx,ry-ly))

    return output
            

# given: a list of initial bboxs
# return: a list of merged bounding boxes
# 
# need to find 2d point clustering and merge the clustering bboxs
# to merge, find xmin, xmax, ymin, ymax of all merging bbox and add margin
def merge_bbox(rects):
    # for i in range(len(rects)):
    #   print(rects[i])
    #first dismiss the rect that is too small
    want = []
    for rect in rects:
      if rect[3] > 0 or rect[2]>0:
        want.append(rect)
    #now seperate them into different clusters, if the rects are within 50 pixels assume they are same cluster
    clusters = []
    if len(want) == 1: 
      clusters.append([want[0]])


    used_rect_list = []
    for i in range(len(want)):
      # print(used_rect_list)
      if not i in used_rect_list:
        temp = []
        temp.append(want[i])
        used_rect_list.append(i)
        for j in range(i+1,len(want)):
          if not j in used_rect_list:
            min_dist = find_min_distance(want[i],want[j])

            ## The distance threshold
            if min_dist < (want[i][2]+want[i][3])/2.5:
              temp.append(want[j])
              used_rect_list.append(j)
        clusters.append(temp)
    #return boxes
    output = []
    for i in range(len(clusters)):
      lx, ly, rx, ry, w, h = 10000, 10000, -10000, -10000, 0, 0
      for j in range(len(clusters[i])):
        if lx > clusters[i][j][0]:
          lx = clusters[i][j][0]
        if ly > clusters[i][j][1]:
          ly = clusters[i][j][1]
        if rx < clusters[i][j][0]+clusters[i][j][2]:
          rx = clusters[i][j][0]+clusters[i][j][2]
        if ry < clusters[i][j][1]+clusters[i][j][3]:
          ry = clusters[i][j][1]+clusters[i][j][3]
      output.append((lx,ly,rx-lx,ry-ly))
    # print(len(output))

    real_output = []
    for a in range(len(output)):
      if output[a][2]*output[a][3] > 1000:

        x_min = max(output[a][0] - 12, 0)
        y_min = max(output[a][1] - 12, 0)
        x_max = min(output[a][0] + output[a][2] + 24, 640)
        y_max = min(output[a][1] + output[a][3] + 24, 480)
        width = x_max - x_min
        height = y_max - y_min

        real_output.append((x_min,y_min,width,height))

    return real_output

def find_wbc_bbox(input_img):
    
    # min area for bbox bounded area
    limit_area = 10
    x = 0   
    y = 0   
    w = 0   
    h = 0   

    img1 = input_img.copy()
    img = cv2.add(img1, 0.70)
    img_3 = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask1 = cv2.inRange(img_3, (90,140,0), (255,255,255))   
    mask2 = cv2.inRange(img_3, (90,90,0), (255,255,255))   
    mask1 = cv2.equalizeHist(mask1)
    mask2 = cv2.equalizeHist(mask2)
    
    mask = mask1 + mask2   
    kernel = np.ones((1,4),np.uint8)   
    mask = cv2.dilate(mask,kernel,iterations = 1)   
    kernel_close = np.ones((3,3),np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel_close)   
    blur2 = cv2.medianBlur(mask,7)   
    canny = cv2.Canny(blur2, 100,200)   
    contours, hierarchy = cv2.findContours(canny,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE) 

    rects = []
    fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(5, 5))

    for cnt in contours:   
        if cv2.contourArea(cnt) >= limit_area:
            x, y, w, h = cv2.boundingRect(cnt)

            rects.append((x, y, w, h))

            roi = blur2[y:y+h, x:x+w]
            image_roi = cv2.resize(roi, (128,128), interpolation=cv2.INTER_AREA)
            image_roi = cv2.medianBlur(image_roi, 5)
            (T, thresh) = cv2.threshold(image_roi, 10, 255, cv2.THRESH_BINARY)
            contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            contours = [i for i in contours if cv2.contourArea(i) <= 5000]
            cv2.fillPoly(thresh, contours, color=(0,0,0))
            for bbox in rects:
                cv2.rectangle(img1, (bbox[0], bbox[1]), (bbox[0] + bbox[2], bbox[1] + bbox[3]), (255,0,0), 3)
    
    ax.imshow(img1)
    # find merged bboxs
    merged_bbox = merge_bbox(rects)

    bbox_count = len(merged_bbox)
    for i in range(bbox_count):
        merged_bbox = merge_by_iou(merged_bbox)



    return merged_bbox

In [0]:
# show original img
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(5, 5))
ax.imshow(imgs[4])

merged_bbox = find_wbc_bbox(imgs[4])
imgg = imgs[4].copy()
fig1, ax1 = plt.subplots(ncols=1, nrows=1, figsize=(5, 5))
for bbox in merged_bbox:
        cv2.rectangle(imgg, (bbox[0], bbox[1]), (bbox[0] + bbox[2], bbox[1] + bbox[3]), (255,0,0), 3)
ax1.imshow(imgg)

In [0]:
for i in sorted(dataset.keys()): 
    if(i in range(200, 410)):
        img = dataset[i][0].copy()
        label_list = dataset[i][1]
        fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(3, 3))
        j = str(i)+ ' ' + label_list[0][0]
        plt.title(j)

        for j in label_list:
            x, y, w, h = j[1]
            cv2.rectangle(img, (x, y), (x+w,y+h), (255,0,0), 3)

        ax.imshow(img)

In [0]:
label_list = []
for i in sorted(dataset.keys()):
    label_list_i = dataset[i][1]
    label_list.append((i, label_list_i))

print(label_list[0])

# Data Processing: Merge Images and Merge Labels

In [0]:
def merge_images(image1, image2, image3, image4):
    """Merge two images into one, displayed side by side
    :param file1: path to first image file
    :param file2: path to second image file
    :return: the merged Image object
    """
    # image1 = Image.open(file1)
    # image2 = Image.open(file2)
    # image3 = Image.open(file3)
    # image4 = Image.open(file4)
    (width, height) = image1.size

    result_width = 2*width
    result_height = 2*height

    result = Image.new('RGB', (result_width, result_height))
    result.paste(im=image1, box=(0, 0))
    result.paste(im=image2, box=(width, 0))
    result.paste(im=image3, box=(0, height))
    result.paste(im=image4, box=(width, height))
    result = result.resize((224, 224))
    return result

def merge_labels(l1, l2, l3, l4):
  merged_label = []
  for i in l1:
    cls = i[0]

    label = list(i[1])
    label = [x/2 for x in label]
    w_ratio = 224/640
    h_ratio = 224/480
    label[0] = label[0]*w_ratio
    label[1] = label[1]*h_ratio
    label[2] = label[2]*w_ratio
    label[3] = label[3]*h_ratio
    label = [round(x) for x in label]
    label = tuple(label)
    merged_label.append((cls,label))
  for i in l2:
    cls = i[0]
    label = list(i[1])
    label = [x/2 for x in label]
    label[0] += 640/2
    w_ratio = 224/640
    h_ratio = 224/480
    label[0] = label[0]*w_ratio
    label[1] = label[1]*h_ratio
    label[2] = label[2]*w_ratio
    label[3] = label[3]*h_ratio
    label = [round(x) for x in label]
    label = tuple(label)
    merged_label.append((cls,label))
  for i in l3:
    cls = i[0]
    label = list(i[1])
    label = [x/2 for x in label]
    label[1] += 480/2
    w_ratio = 224/640
    h_ratio = 224/480
    label[0] = label[0]*w_ratio
    label[1] = label[1]*h_ratio
    label[2] = label[2]*w_ratio
    label[3] = label[3]*h_ratio
    label = [round(x) for x in label]
    label = tuple(label)
    merged_label.append((cls,label))
  for i in l4:
    cls = i[0]
    label = list(i[1])
    label = [x/2 for x in label]
    label[0] += 640/2
    label[1] += 480/2
    w_ratio = 224/640
    h_ratio = 224/480
    label[0] = label[0]*w_ratio
    label[1] = label[1]*h_ratio
    label[2] = label[2]*w_ratio
    label[3] = label[3]*h_ratio
    label = [round(x) for x in label]
    label = tuple(label)
    merged_label.append((cls,label))
  return merged_label

In [0]:
def generate_rand_image(src_path, start, end):
  while True:
    num = random.randint(start, end)
    s = '%0*d' % (3, num)
    image_path = src_path + "/BloodImage_00" + s+ ".jpg"
    try:
      image = Image.open(image_path)
      return image, num
    except FileNotFoundError:
      # print("Can't find this path: {}, generate a new one")
      continue
  print("we shouldn't get here")
  return None


def get_bbox_list(idx):
  csv_path = '/content/gdrive/My Drive/APS360/APS360 Final Project/Dataset/dataset1/bbox2.csv'
  with open(csv_path) as f:
    f_reader = csv.reader(f, delimiter=',')
    for row in f_reader:
      if row[0] == str(idx):
        ret = list(ast.literal_eval(row[1]))
        return ret
    else:
      print("so bad, idx is {}".format(idx))

def get_merged_list(idx):
  csv_path = '/content/gdrive/My Drive/APS360/APS360 Final Project/Dataset/dataset1/Merged Labels/ValidationLabels.csv'
  with open(csv_path) as f:
    f_reader = csv.reader(f, delimiter=',')
    for row in f_reader:
      if row[0] == str(idx):
        ret = list(ast.literal_eval(row[1]))
        return ret
    else:
      print("so bad, idx is {}".format(idx))

def build_merged_dataset(s):
  src_path = '/content/gdrive/My Drive/APS360/APS360 Final Project/Dataset/dataset1/JPEGImages'
  dest_path = '/content/gdrive/My Drive/APS360/APS360 Final Project/Dataset/dataset1/Merged Images2/'+s

  #add this to create list of info
  info_list = []
  start = 0
  end = 0
  #count for each class
  if s=="Train":
    num = 1000
    start = 0
    end = 200
  elif s=="Validation":
    num = 500
    start = 201
    end = 301
  else:
    num = 500
    start = 302
    end = 410
  for i in range(0, num):
    image1,idx1 = generate_rand_image(src_path, start, end)
    image2,idx2 = generate_rand_image(src_path, start, end)
    image3,idx3 = generate_rand_image(src_path, start, end)
    image4,idx4 = generate_rand_image(src_path, start, end)
    l1 = get_bbox_list(idx1)
    l2 = get_bbox_list(idx2)
    l3 = get_bbox_list(idx3)
    l4 = get_bbox_list(idx4)
    merged_image = merge_images(image1, image2, image3, image4)
    merged_label = merge_labels(l1, l2, l3, l4)
    # plot_img_with_bbx(merged_image, merged_label)

    #append merged image to info list
    merged_label.sort(key=lambda a: a[0])
    info_list.append(merged_label)

    merged_path = dest_path+"/merged_"+str(i)+".jpg"
    merged_image.save(merged_path)
  return info_list

def plot_img_with_bbx(img, label_list):
  img_copy = np.array(img.copy())
  fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(3,3))
  for i in label_list:
    x,y,w,h = i[1]
    cv2.rectangle(img_copy,(x,y),(x+w,y+h),(255,0,0),3)
  ax.imshow(img_copy)

def count_class(info_list):
    class_table = {} #creates a dictionary

    for items in info_list:
        if items[0] not in class_table:
            class_table[items[0]] = 1
        else:
            class_table[items[0]] +=1

    return class_table

def create_label_csv(info_list, s):
    path = '/content/gdrive/My Drive/APS360/APS360 Final Project/Dataset/dataset1/Merged Labels2/'
    name = s+'Labels.csv'
    with open(path + name, 'w', newline='') as file:
        field_names = ['index', 'class_info', 'class_count']
        writer = csv.DictWriter(file, fieldnames=field_names)
        writer.writeheader()
        index_num = 0
        for items in info_list:
            writer.writerow({'index': index_num, 'class_info': items, 'class_count': count_class(items)})
            index_num+=1

# Data Processing: Build Train, Validation and Test Dataset for R-CNN Model

In [0]:
train_info_list = build_merged_dataset("Train") create_label_csv(train_info_list, "Train")

val_info_list = build_merged_dataset("Validation") create_label_csv(val_info_list, "Validation")

test_info_list = build_merged_dataset("Test") create_label_csv(test_info_list, "Test")

# Baseline Model - KNN

In [0]:
!pip install imutils

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from imutils import paths
import numpy as np
import argparse
import imutils
import cv2
import os

In [0]:
def image_to_feature_vector(image, size=(32, 32)):
	# resize the image to a fixed size, then flatten the image into
	# a list of raw pixel intensities
	return cv2.resize(image, size).flatten()

In [0]:
def extract_color_histogram(image, bins=(8, 8, 8)):
	# extract a 3D color histogram from the HSV color space using
	# the supplied number of `bins` per channel
	hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
	hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,
		[0, 180, 0, 256, 0, 256])
	# handle normalizing the histogram if we are using OpenCV 2.4.X
	if imutils.is_cv2():
		hist = cv2.normalize(hist)
	# otherwise, perform "in place" normalization in OpenCV 3 (I
	# personally hate the way this is done
	else:
		cv2.normalize(hist, hist)
	# return the flattened histogram as the feature vector
	return hist.flatten()

In [0]:
# loop over the input images
for (i, imagePath) in enumerate(imagePaths):
  
	image = cv2.imread(imagePath)
	label = imagePath.split(os.path.sep)[-1].split(".")[0]
	# extract raw pixel intensity "features", followed by a color
	# histogram to characterize the color distribution of the pixels
	# in the image
	pixels = image_to_feature_vector(image)
	hist = extract_color_histogram(image)
	# update the raw images, features, and labels matricies,
	# respectively
	rawImages.append(pixels)
	features.append(hist)
	labels.append(label)
	# show an update every 1,000 images
	if i > 0 and i % 1000 == 0:
		print("[INFO] processed {}/{}".format(i, len(imagePaths)))

In [0]:
# partition the data into training and testing splits, using 75%
# of the data for training and the remaining 25% for testing
(trainRI, testRI, trainRL, testRL) = train_test_split(
	rawImages, labels, test_size=0.25, random_state=42)
(trainFeat, testFeat, trainLabels, testLabels) = train_test_split(
	features, labels, test_size=0.25, random_state=42)

In [0]:
print("[INFO] evaluating raw pixel accuracy...")
model = KNeighborsClassifier(n_neighbors=args["neighbors"],
	n_jobs=args["jobs"])
model.fit(trainRI, trainRL)
acc = model.score(testRI, testRL)
print("[INFO] raw pixel accuracy: {:.2f}%".format(acc * 100))

In [0]:
# train and evaluate a k-NN classifer on the histogram
# representations
print("[INFO] evaluating histogram accuracy...")
model = KNeighborsClassifier(n_neighbors=args["neighbors"],
	n_jobs=args["jobs"])
model.fit(trainFeat, trainLabels)
acc = model.score(testFeat, testLabels)
print("[INFO] histogram accuracy: {:.2f}%".format(acc * 100))

# Generating Regional Proposal

In [0]:
!pip install selectivesearch

In [0]:
import selectivesearch
import skimage.io
import skimage
import matplotlib.patches as mpatches

In [0]:
def clip_pic(img, padding,x_min, y_min, x_max, y_max):
    x_min = x_min - padding
    y_min = y_min - padding
    x_min = max(0, x_min)
    y_min = max(0, y_min)

    x_max = x_max + padding
    y_max = y_max + padding
    x_max = min(x_max, 224)
    y_max = min(y_max, 224)
    return img[y_min:y_max, x_min:x_max]

In [0]:
def area_inter_val(xmin_a, xmax_a, ymin_a, ymax_a, xmin_b, xmax_b, ymin_b, ymax_b):
    
    intersect= not ( xmin_b > xmax_a or xmax_b < xmin_a or ymax_b < ymin_a or ymin_b > ymax_a)

    if intersect == True:
        x_sorted_list = sorted([xmin_a, xmax_a, xmin_b, xmax_b])
        y_sorted_list = sorted([ymin_a, ymax_a, ymin_b, ymax_b])
        x_intersect_w = x_sorted_list[2] - x_sorted_list[1]
        y_intersect_h = y_sorted_list[2] - y_sorted_list[1]
        area_inter = x_intersect_w * y_intersect_h
        return area_inter
    else:
        return 0

def iou_val(xmin_a, xmax_a, ymin_a, ymax_a, xmin_b, xmax_b, ymin_b, ymax_b):
    area_inter = area_inter_val(xmin_a, xmax_a, ymin_a, ymax_a, xmin_b, xmax_b, ymin_b, ymax_b)
    if area_inter > 0:
        area_1 = (xmax_a - xmin_a) * (ymax_a - ymin_a)
        area_2 = (xmax_b - xmin_b) * (ymax_b - ymin_b)
        return float(area_inter) / (area_1 + area_2 - area_inter)
    else:
        return 0

def proposal_bbox(image_sample):
    _, regions = selectivesearch.selective_search(image_sample, scale=15, sigma=0.9, min_size=80)
    candidates = set()
    for r in regions:
        x, y, w, h = r['rect']
        if r['rect'] in candidates:
            continue
        if r['size'] < 50:
            continue
        if w == 0 or h == 0:
            continue
        candidates.add(r['rect'])
    return list(candidates)

#[('NEUTROPHIL', (285, 169, 221, 206))]

def regional_proposal(image_sample, proposal_bboxes):
    proposals = []
    for i in proposal_bboxes:
        x, y, w, h = i
        img_proposal = clip_pic(image_sample, 16, x, y, x+w, y+h)
        img_proposal2 = skimage.transform.resize(img_proposal, (224, 224))
        proposals.append((img_proposal2, (x,y,w,h)))
    return proposals

In [0]:
## save the image into folders
background_path = '/content/gdrive/My Drive/APS360/Train_Proposals/BACKGROUND/'
eo_path = '/content/gdrive/My Drive/APS360/Train_Proposals/EOSINOPHIL/'
lymph_path = '/content/gdrive/My Drive/APS360/Train_Proposals/LYMPHOCYTE/'
mono_path = '/content/gdrive/My Drive/APS360/Train_Proposals/MONOCYTE/'
neut_path = '/content/gdrive/My Drive/APS360/Train_Proposals/NEUTROPHIL/'
name = 'image_'


# generate proposal for 100 train images
all_proposals = []
for i in range(100):
    image_sample = train_imgs[i].copy()
    bbox = proposal_bbox(image_sample)
    proposals = regional_proposal(image_sample, bbox, train_labels[i])
    all_proposals.extend(proposals)

cnt = 0
for j in range(len(all_proposals)):
    img_class = all_proposals[j][1]
    if img_class== 'BACKGROUND':
        cnt += 1

        # select a portion from backgrounds
        if cnt % 20 == 0:
            matplotlib.image.imsave(background_path + name + str(j) + ext, all_proposals[j][0])
    elif img_class == 'NEUTROPHIL':
        matplotlib.image.imsave(neut_path + name + str(j) + ext, all_proposals[j][0])
    elif img_class == 'EOSINOPHIL':
        matplotlib.image.imsave(eo_path + name + str(j) + ext, all_proposals[j][0])
    elif img_class == 'LYMPHOCYTE':
        matplotlib.image.imsave(lymph_path + name + str(j) + ext, all_proposals[j][0])
    elif img_class == 'MONOCYTE':
        matplotlib.image.imsave(mono_path + name + str(j) + ext, all_proposals[j][0])
    else:
        print('err')

# Transfer Learning and Training

In [0]:
import os
import time
import torchvision.models

# load pretrained alexnet
model = torchvision.models.alexnet(pretrained=True).cuda()

classes = ['EOSINOPHIL', 'MONOCYTE', 'NEUTROPHIL', 'LYMPHOCYTE', 'BACKGROUND' ]

In [0]:
model.classifier[6] = nn.Linear(4096, 5).cuda()
nn.init.xavier_normal_(model.classifier[6].weight)

In [0]:
transform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])

t_dataset = torchvision.datasets.ImageFolder('/content/gdrive/My Drive/APS360/train_proposal-20200405T193852Z-001.zip (Unzipped Files)/train_proposal/', transform=transform)
evens = list(range(0, len(t_dataset), 2))
train_dataset = torch.utils.data.Subset(t_dataset, evens)
print(len(train_dataset))

v_dataset = torchvision.datasets.ImageFolder('/content/gdrive/My Drive/APS360/val_proposal-20200405T195313Z-001.zip (Unzipped Files)/val_proposal/', transform=transform)
evens = list(range(0, len(v_dataset), 2))
val_dataset = torch.utils.data.Subset(v_dataset, evens)
print(len(val_dataset))


batch_size = 512
num_workers = 1

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)

In [0]:
def get_accuracy_loss(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    total_loss = 0.0
    i = 0
    criterion = nn.CrossEntropyLoss()    
    with torch.no_grad():
        batch = 0
        for imgs, labels in data_loader:

            if use_cuda and torch.cuda.is_available():
                imgs = imgs.cuda()
                labels = labels.cuda()

            output = model(imgs)
            loss = criterion(output, labels)
            total_loss += loss.item()
            #select index with maximum prediction score
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(labels.view_as(pred)).sum().item()
            total += imgs.shape[0]
            i+=1
            batch+=1
    final_loss = float(total_loss) / (i + 1)
    return correct / total, final_loss

In [0]:
use_cuda = True

def train_model(model, train_loader, val_loader, num_epochs=1, learn_rate = 0.001):

    torch.manual_seed(1000)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learn_rate)

    train_acc, val_acc, train_loss, val_loss = [], [], [], []

    # training
    print ("Training Started...")
    n = 0 # the number of epochs
    for epoch in range(num_epochs):
        start = time.time()
        j = 0
        model.train()
        for imgs, labels in iter(train_loader):
            
            if use_cuda and torch.cuda.is_available():
              imgs = imgs.cuda()
              labels = labels.cuda()
            out = model(imgs)             # forward pass
            loss = criterion(out, labels) # compute the total loss
            loss.backward()               # backward pass (compute parameter updates)
            optimizer.step()              # make the updates for each parameter
            optimizer.zero_grad()         # a clean up step for PyTorch
            j +=1
            print(j, end=' ')
            
        n += 1
        # track accuracy
        t_acc, t_loss = get_accuracy_loss(model, train_loader)
        v_acc, v_loss = get_accuracy_loss(model, val_loader)

        train_acc.append(t_acc)
        train_loss.append(t_loss)
        val_acc.append(v_acc)
        val_loss.append(v_loss)

        print('')
        print("epoch:", epoch, end = " ")
        print("train loss:", train_loss[-1], end = " | ")
        print("train acc:", train_acc[-1], end = " | ")
        print("val loss:", val_loss[-1], end = " | ")
        print("val acc:", val_acc[-1])
        print('')
        
        end = time.time()
        print('time took: ', end-start)
    
    n = len(train_acc)
    plt.figure(0)
    plt.plot(range(1,n+1), train_loss, label="Train")
    plt.plot(range(1,n+1), val_loss, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(loc='best')

    plt.figure(1)
    plt.plot(range(1,n+1), train_acc, label="Train")
    plt.plot(range(1,n+1), val_acc, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend(loc='best')
            
    return train_acc, val_acc



In [0]:
train_model(model=model, train_loader=train_loader, val_loader=val_loader, num_epochs = 5, learn_rate = 0.0004)

In [0]:
torch.save(model.state_dict(), '/content/gdrive/My Drive/APS360/alexnet2.pt')

In [0]:
state = torch.load('/content/gdrive/My Drive/APS360/alexnet.pt')
model.load_state_dict(state)

# Testing

In [0]:
transform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])

test_dataset = torchvision.datasets.ImageFolder('/content/gdrive/My Drive/APS360/test_proposal-20200405T193546Z-001.zip (Unzipped Files)/test_proposal', transform=transform)
evens = list(range(0, len(test_dataset), 2))

batch_size = 512
num_workers = 1

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)

In [0]:
acc, loss = get_accuracy_loss(model=model, data_loader=test_loader)
print('Test Accuracy: ', acc)

In [0]:
def get_wbc_counts(image):

    # generate regional proposals
    image_sample = image.copy()
    proposals = regional_proposal(image_sample, bbox)
    for i in range(5):
        fig3, ax3 = plt.subplots(ncols=1, nrows=1, figsize=(4, 4))
        ax3.imshow(proposals[i][0])

    classes = ['MONOCYTE','LYMPHOCYTE', 'EOSINOPHIL', 'NEUTROPHIL', 'BACKGROUND']

    # get output for each proposal
    types = {}
    types['EOSINOPHIL'] = []
    types['LYMPHOCYTE'] = []
    types['MONOCYTE'] = []
    types['NEUTROPHIL'] = []
    for i in proposals:
        x = torch.tensor(i[0].reshape(-1, 3, 224, 224)).cuda()     # if numpy array
        x = x.float()
        out = model(x)
        out_class = out.max(1, keepdim=True)[1]
        if out_class != 'BACKGROUND':
            types[classes[out_class]].append(i[1]) # append its bbox to that category

    # merge proposals if iou > 0.5, same label
    merged = {}
    for c in types.keys():
        bbox_list = types[c]
        merged_bboxes1 = merge_bbox(bbox_list)
        merged_bboxes = merge_by_iou(merged_bboxes1)
        merged[c] = merged_bboxes

    return merged


In [0]:
test_sample = imread('/content/gdrive/My Drive/APS360/merged_378.jpg')
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(3, 3))
ax.imshow(test_sample)
get_wbc_counts(test_sample)

In [0]:
def confusion_matrix(model,data_loader):
    matrix = np.zeros(shape=(4,4))

    n = 0

    for imgs, labels in data_loader:
        
        if use_cuda and torch.cuda.is_available():
          imgs = imgs.cuda()
          labels = labels.cuda()

        output = model(imgs)
        pred = output.max(1, keepdim=True)[1]
        for i in range(pred.shape[0]):
            matrix[labels[i]][pred[i]] +=1
        n+=1
        print(n)

    return matrix